In [1]:
from __future__ import unicode_literals, print_function, division
from io import open
from os import system
from dataloader import *
from VAE import *
from scores import *

import unicodedata
import string
import re
import random
import time
import math
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
import matplotlib.ticker as ticker
import matplotlib.pyplot as plt
plt.switch_backend('agg')

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
#----------Hyper Parameters----------#
hidden_size = 256
vocab_size = 28 #The number of vocabulary
teacher_forcing_ratio = 1.0
empty_input_ratio = 0.1
KLD_weight = 0.0
LR = 0.05

SOS_token = 0
EOS_token = vocab_size-1

# Prepare data

In [4]:
train_vocab = load_data('./data/train.txt')
test_vocab = load_data('./data/test.txt')

## Get different tense pairs for (unconditional) VAE training

In [14]:
def get_tense_paris(train_vocab, source_index, target_index):
    pairs = []

    for vocabs in train_vocab:
        pairs.append((vocabs[source_index],vocabs[target_index]))
        
    return pairs

### Simple Present -> Third Person

In [15]:
train_st_tp  = get_tense_paris(train_vocab, 0, 1)

### Simple Present -> Present Progressive

In [16]:
train_st_pp  = get_tense_paris(train_vocab, 0, 2)

### Simple Present -> Past

In [17]:
train_st_past  = get_tense_paris(train_vocab, 0, 3)

# Train VAE

In [ ]:
def tensorFromPairs(pair):
    pass

In [ ]:
def train(vae_model, n_iters, print_every=1000, learning_rate=0.01):
    
    for i in range(n_iters):
        # Initialize hidden feature
        hidden = torch.zeros(1, 1, hidden_size, device=device)
        
        